In [ ]:
'''Adapted from https://github.com/chrisvdweth/nus-cs4248x/blob/master/3-neural-nlp/Section%204.2%20-%20Transformer%20Machine%20Translation.ipynb'''
!pip install -U spacy
!pip install -U torchdata
!pip install datasets
!pip install portalocker
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00


In [ ]:
import math
from tqdm.notebook import tqdm
from typing import Iterable, List
from timeit import default_timer as timer
import torch
import torch.nn as nn
import pandas as pd
from torch import Tensor
from torch.nn import Transformer
import sentencepiece as spm

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k

In [ ]:
use_cuda = torch.cuda.is_available()

# Use this line below to enforce the use of the CPU
# use_cuda = False

DEVICE = torch.device("cuda:0" if use_cuda else "cpu")

print("Available device: {}".format(DEVICE))

Available device: cuda:0


In [ ]:
from datasets import load_dataset
train_dataset = load_dataset(
                  "ngxingyu/iwslt17_google_trans_scores_sentiments",
                  # streaming=True,
                  split="train",
                )
test_dataset = load_dataset(
                  "ngxingyu/iwslt17_google_trans_scores_sentiments",
                  # streaming=True,
                  split="test",
                )
validation_dataset = load_dataset(
                    "ngxingyu/iwslt17_google_trans_scores_sentiments",
                    # streaming=True,
                    split="validation",
                )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/229736 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/875 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8549 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset, validation_dataset)

Dataset({
    features: ['bleurt_score', 'comet_score', 'en', 'google_zh', 'zh', 'en_sentiment', 'zh_sentiment'],
    num_rows: 229736
}) Dataset({
    features: ['bleurt_score', 'comet_score', 'en', 'google_zh', 'zh', 'en_sentiment', 'zh_sentiment'],
    num_rows: 875
})


In [ ]:
SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'zh'

# Placeholders
token_transform = {}
vocab_transform = {}

Here we are using **SPM Unigram** Tokenisation

In [ ]:
en_data = train_dataset[SRC_LANGUAGE]
zh_data = train_dataset[TGT_LANGUAGE]
data = []
with tqdm(total=len(train_dataset)) as progress_bar:
  for i in range(len(train_dataset)):
      parallel_sentences = {}
      parallel_sentences[SRC_LANGUAGE] = en_data[i]
      parallel_sentences[TGT_LANGUAGE] = zh_data[i]
      data.append(parallel_sentences)
      progress_bar.update(1)

# Concatenate English and Chinese sentences
all_sentences = [entry[SRC_LANGUAGE] for entry in data] + [entry[TGT_LANGUAGE] for entry in data]
# Save the sentences to a text file
with open("all_sentences.txt", "w", encoding="utf-8") as f:
    for sentence in all_sentences:
        f.write(sentence + "\n")

# Train the SentencePiece model
spm.SentencePieceTrainer.train(input="all_sentences.txt", model_prefix="your_model", vocab_size=10000)

  0%|          | 0/229736 [00:00<?, ?it/s]

In [ ]:
# Load SentencePiece model
tokeniser = spm.SentencePieceProcessor()
tokeniser.load("your_model.model")

# Test tokeniser
entry = data[0]
english_tokens = tokeniser.encode_as_ids(entry['en'])
chinese_tokens = tokeniser.encode_as_pieces(entry['zh'])
print("English tokens:", english_tokens)
print("Chinese tokens:", chinese_tokens)

English tokens: [1090, 19, 63, 237, 3, 2971, 5, 24, 20, 11, 6, 4438, 13, 453, 6676, 10, 38, 7, 2560, 10, 340, 10, 25, 2675, 5694, 239, 16, 11, 54, 3958, 8859, 5]
Chinese tokens: ['▁', '非常', '谢谢', ',', '克里斯', '。', '的确', '非常', '荣幸', '▁能', '有', '第二', '次', '站在', '这个', '台', '上', '的机会', ',', '我', '真是', '非常', '感', '激', '。']


In [ ]:
from torch.utils.data import Dataset
from typing import Iterable, List

class CustomDataset(Dataset):
    def __init__(self, data_iter: Iterable, tokeniser):
        self.data_iter = data_iter
        self.tokeniser = tokeniser
        self.process_data_iter()
        self.tokens = {SRC_LANGUAGE: [], TGT_LANGUAGE: []}

    def __len__(self):
        return len(self.data_iter)

    def __getitem__(self, idx):
        sample = self.data_iter[idx]
        return sample

    def process_data_iter(self):
        en_data = self.data_iter['en']
        zh_data = self.data_iter['zh']
        iter = []
        for i in range(len(self.data_iter)):
          data_sample = {}
          data_sample['en'] = en_data[i]
          data_sample['zh'] = zh_data[i]
          iter.append(data_sample)
        self.data_iter = iter

    def tokenize_sentence(self, sentence: str, language: str):
        tokens = self.tokeniser.encode_as_pieces(sentence)
        self.tokens[language].append(tokens)
        return tokens


    def yield_tokens(self, language: str):
        with tqdm(total=len(self.data_iter)) as progress_bar:
            for data_sample in self.data_iter:
                yield self.tokenize_sentence(data_sample[language], language)
                progress_bar.update(1)

In [ ]:
from torchtext.datasets import multi30k, Multi30k
custom_train_dataset = CustomDataset(train_dataset, tokeniser)
custom_validation_dataset = CustomDataset(validation_dataset, tokeniser)
custom_test_dataset = CustomDataset(test_dataset, tokeniser)

# Define special symbols and indices
PAD_IDX, UNK_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<PAD>', '<UNK>', '<SOS>', '<EOS>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(custom_train_dataset.yield_tokens(ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set `UNK_IDX` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)

  0%|          | 0/229736 [00:00<?, ?it/s]

  0%|          | 0/229736 [00:00<?, ?it/s]

In [ ]:
print('EN vocab length:', len(vocab_transform[SRC_LANGUAGE]))
print('ZH vocab length:', len(vocab_transform[TGT_LANGUAGE]))

EN vocab length: 3881
ZH vocab length: 9611


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])


class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [ ]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask)

In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask.bool()


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)

    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
# Custom transformation function for tokenization using SentencePieceProcessor
def tokenize_with_sentencepiece(tokenizer):
    def transform(txt_input):
        return tokenizer.encode_as_pieces(txt_input)
    return transform

# Helper method to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# method to add SOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([SOS_IDX]), torch.tensor(token_ids), torch.tensor([EOS_IDX])))


# `src` and `tgt` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(tokenize_with_sentencepiece(tokeniser), # Tokenization
                                               vocab_transform[ln], # Vectorization
                                               tensor_transform)    # Add SOS/EOS and create tensor


# Method to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for i in range(len(batch)):
        src_sample, tgt_sample = batch[i][SRC_LANGUAGE], batch[i][TGT_LANGUAGE]
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [ ]:
def train_epoch(model, optimizer, criterion):
    model.train()
    losses = 0

    train_dataloader = DataLoader(custom_train_dataset.data_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in tqdm(train_dataloader, total=len(list(train_dataloader))):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        # Remove last entry an all target sequences (typically PAD, can be EOS)
        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        # Remove <SOS> from all targets
        tgt_out = tgt[1:, :]

        loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    val_dataloader = DataLoader(custom_validation_dataset.data_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in tqdm(val_dataloader, total=len(list(val_dataloader))):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]

        loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [ ]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 32
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

# Create model
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

# Initialize weights
for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

# Move model to device (ideally GPU, otherwise CPU)
transformer = transformer.to(DEVICE)

# Define loss function
criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# Define optimizer
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
NUM_EPOCHS = 2
best_val_loss = 1e5

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer, criterion)
    end_time = timer()
    val_loss = evaluate(transformer)

    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time (total) = {(end_time - start_time):.3f}s"))

  0%|          | 0/28717 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

Epoch: 1, Train loss: 6.255, Val loss: 6.241, Epoch time (total) = 902.591s


  0%|          | 0/28717 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

Epoch: 2, Train loss: 5.556, Val loss: 5.691, Epoch time (total) = 904.779s


In [ ]:
# Method to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

# Actual method to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model, src, src_mask, max_len=num_tokens+5, start_symbol=SOS_IDX).flatten()

    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<SOS>", "").replace("<EOS>", "")

def translate_dataset(model, validation_dataset):
    translations = []
    for sample in tqdm(validation_dataset, total=len(validation_dataset)):
        src_sentence = sample[SRC_LANGUAGE]
        tgt_sentence = sample[TGT_LANGUAGE]

        # Translate the source sentence using the model
        translated_sentence = translate(model, src_sentence)

        # Append the translation along with source and target sentences
        translations.append({'en': src_sentence, 'zh': tgt_sentence, 'translated_output': translated_sentence})

    return pd.DataFrame(translations)

In [ ]:
print(translate(transformer, "Thank you very much."))
print(translate(transformer, "And it was a huge success."))

 ▁谢谢 各位 
 ▁这 让我 感到 兴奋 


In [ ]:
translations = translate_dataset(transformer, custom_test_dataset.data_iter)

  0%|          | 0/8549 [00:00<?, ?it/s]

In [ ]:
def clean_tranlsations(text):
    # Replace underscores with spaces
    cleaned_text = text.replace("▁", " ")
    # For Chinese text, trim leading and trailing spaces, but preserve spaces between characters
    if text.strip() == text:  # Check if the text has no leading or trailing spaces
        cleaned_text = cleaned_text.strip()  # Trim leading and trailing spaces
    return cleaned_text

translations['translated_output'] = translations['translated_output'].apply(clean_tranlsations)
translations.to_csv(f'transformer_spm_base_{BATCH_SIZE}.csv', index=False)

In [ ]:
translations.head()

,en,zh,translated_output
0,"Several years ago here at TED, Peter Skillman ...",几年前，在TED大会上， Peter Skillman 介绍了一个设计挑战 叫做“棉花糖挑战”,"在 去年 , 我 第一次 问 一个 新 西 兰 的 文化 文化 的 文化"
1,And the idea's pretty simple: Teams of four h...,是个非常简单的主意 要求一组四人的团队搭建一个独立的最高建筑 材料是20根意大利面条 一段胶...,说 实 话 说 的是 : 两 位 的 女性 都有 一 大 两 次 : 其 ...
2,The marshmallow has to be on top.,棉花糖必须放在最上面,海 水 下 的 水
3,"And, though it seems really simple, it's actua...",这虽然看似简单，其实并不容易 因为它要求人们 迅速地合作,"现在 , 这 很 有意思 , 因为 人们 知道 他们 会 这样 想 让 很多人 在..."
4,"And so, I thought this was an interesting idea...",我觉得这是个有趣的主意 我把它放到了设计专题讨论会上,"所以我 想 , 我 喜欢 这个 叫做 “ 艺术 ” , 叫做 “ 艺术 ”"
